<a href="https://colab.research.google.com/github/yijhenjhen/pythonProject/blob/master/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 3s (10.7 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u312-b07-0ubunt

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import lower, length, size
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

In [3]:
spark

In [4]:
sc

<SparkContext master=local appName=pyspark-shell>

In [5]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [9]:
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/oriDataTag.csv",
                      inferSchema=True,
                      header=True,
                      encoding='utf-8')

# get the dataset
def get_dataset(data):
	
    # Preprocessing and feature engineering
    feature_prep_ori = data.select('sex', 'job', 'age', 'market', 'risk', 'joinTime', 'content', 'tag').orderBy('tag')
    feature_prep = feature_prep_ori.dropna(how='any', thresh=None, subset=None)
    feature_prep = StringIndexer(inputCol="sex", outputCol="SexIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="job",outputCol="jobIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="joinTime",outputCol="joinTimeIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="risk",outputCol="riskIndex").fit(feature_prep).transform(feature_prep)

    data = feature_prep.toPandas()
    data.to_csv('/content/drive/MyDrive/Colab Notebooks/dataTrain.csv', sep=',', encoding='utf_8_sig', index=True)
    dataTrain=data.sample(frac=0.9)
    dataTest=data[~data.index.isin(dataTrain.index)]
    dataframe = dataTrain[['SexIndex', 'jobIndex', 'age', 'riskIndex', 'joinTimeIndex', 'tag']]


    # 建立label的numpy array
    featuresArray = []
    labelArray = []
    df = dataframe['tag']
    k=0

    for i in df:
        index = (df.index)[k]
        featuresVec = [0] * 5
        featuresVec[0] = dataframe.loc[index, 'SexIndex']
        featuresVec[1] = dataframe.loc[index, 'jobIndex']
        featuresVec[2] = dataframe.loc[index, 'age']
        featuresVec[3] = dataframe.loc[index, 'riskIndex']
        featuresVec[4] = dataframe.loc[index, 'joinTimeIndex']
        featuresArray.append(np.array(featuresVec))

        tagVec = [0] * 13
        for j in i.split(','):

            try:
                j = int(j)
                if j < 14:
                    tagVec[j-1] = 1
                      #data.loc[index, 'tag'] = str(tagVec)
                else:
                    tagVec[j-1] = 1
                      #data.loc[index, 'tag'] = str(tagVec)
            
            # print(np.array(tagVec))
            except:
                pass
        labelArray.append(np.array(tagVec))
        k+=1

    featuresArray = np.array(featuresArray)
    labelArray = np.array(labelArray)


    # summarize dataset shape
    print(featuresArray.shape, labelArray.shape)
    # summarize first few examples
    for i in range(10):
        print(featuresArray[i], labelArray[i])

    return featuresArray, labelArray, dataTest

In [10]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(128, input_dim=n_inputs, activation='relu'))# kernel_initializer='he_uniform', 
    # model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))# kernel_initializer='he_uniform',
    # model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    # model.add(Dropout(0.2))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [11]:
# evaluate a model using repeated k-fold cross-validation
from sklearn.utils import class_weight
def evaluate_model(X, y):

	results = list()
	
	time=0
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	print(X.shape[1], y.shape[1])

	# define evaluation procedure
	# cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
	

	# for i in range(epoch):
	# 	model = get_model(n_inputs, n_outputs)
	# 	model.fit(X_train, y_train, verbose=2, epochs=100)
	# 	yhat = model.predict(X_test)
	# 	yhat = yhat.round()
	# 	acc = accuracy_score(y_test, yhat)
	# 	print('>%.3f' % acc)
	# 	results.append(acc)
	
	# enumerate folds
	# y_pre = np.zeros((179, 13))
	model = get_model(n_inputs, n_outputs)

	class_weight = {0:1.87, 1:4.51, 2:0.4, 3:1.8, 4:1.81, 5:0.5, 6:5,
	         7:2.99, 8:1.81, 9:8, 10:8, 11:4.2, 12:2.28}
	model.fit(X_train, y_train, verbose=0, epochs=1000)
 
	yhat = model.predict(X_test)
	# for train_ix, test_ix in cv.split(X):
	# 	# prepare data
	# 	X_train, X_test = X[train_ix], X[test_ix]
	# 	y_train, y_test = y[train_ix], y[test_ix]
	# 	# define model
	# 	model = get_model(n_inputs, n_outputs)
	# 	# fit model
	# 	model.fit(X_train, y_train, verbose=0, epochs=10)
	# 	# make a prediction on the test set
	# 	yhat = model.predict(X_test)
		
		# round probabilities to class labels
		# for yn in range(len(yhat)):
		# 	index=0
		# 	# for iyn in yhat[yn]:
				
		# 		# print(iyn)
		# 	print(yhat[yn].round(4))
				# index += 1
		# print('#####################################################################################')
		# yhat = yhat.round()
		# # print(yhat.round(4))
		# # calculate accuracy
		# acc = accuracy_score(y_test, yhat)
		# # store result
		# print('>%.3f' % acc)
		# results.append(acc)
		# #y_pre.append(yhat)
		# y_pre+=yhat
		# time+=1
	#y_pre=y_pre/time
	# for i in range(len(yhat)):
	# 	print(yhat[i])

	
		# print(yhat[i].round(4))
	model.save("/content/drive/MyDrive/Colab Notebooks/modelTag_V3.h5")
	return results

In [12]:
# load dataset
X, y, dataset = get_dataset(data)
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

(1608, 5) (1608, 13)
[ 0.  7. 44.  0.  2.] [0 0 1 0 0 0 0 0 0 0 1 0 1]
[ 0.  9. 33.  0.  0.] [0 0 0 0 0 0 0 0 1 0 1 0 0]
[ 0. 20. 38.  0.  0.] [1 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0. 16. 24.  0.  0.] [1 0 0 0 0 0 0 0 1 0 0 0 0]
[ 0.  6. 41.  2.  2.] [0 0 0 0 0 1 0 0 0 0 0 0 0]
[ 0. 11. 40.  1.  2.] [0 0 0 1 0 0 0 0 0 0 0 0 1]
[ 0.  0. 37.  0.  3.] [0 0 0 0 0 0 0 0 0 0 0 0 1]
[ 0.  6. 48.  0.  4.] [0 0 0 0 1 0 0 0 0 0 0 0 0]
[ 0.  6. 47.  0.  1.] [0 0 0 0 0 0 0 0 0 0 0 1 0]
[ 0.  0. 42.  0.  0.] [0 1 0 0 0 0 0 0 0 1 0 0 0]
5 13
Accuracy: nan (nan)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
from tensorflow.keras.models import load_model
import random
pd.options.display.max_columns = None
test = list()


X, y, dataset = get_dataset(data)
dataset.reset_index(inplace=True, drop=False)
for t in range(178):
    x = random.randrange(0, 178)
    test.append(x)


dataframe = dataset[['SexIndex', 'jobIndex', 'age', 'riskIndex', 'joinTimeIndex', 'tag']]

model = load_model('/content/drive/MyDrive/Colab Notebooks/modelTag_1.h5',compile=False)
acc=0
for i in test:
    # print(i)

    # index = (dataset['index'].index)[0]
    featuresVec = [0] * 5
    featuresVec[0] = dataset.loc[i, 'SexIndex']
    featuresVec[1] = dataset.loc[i, 'jobIndex']
    featuresVec[2] = dataset.loc[i, 'age']
    featuresVec[3] = dataset.loc[i, 'riskIndex']
    featuresVec[4] = dataset.loc[i, 'joinTimeIndex']

    input = np.array(featuresVec).reshape((1,5))
    y_predicted = model.predict(input)
    # print(y_predicted.round(4))
    # print(dataset[i:i+1])

    resh = y_predicted[0]
    top_k = 3
    top_k_idx = resh.argsort()[::-1][0:top_k]
    top_k_idx += 1
    # print("困擾類型:", top_k_idx)
    right=0

    nums=dataset.loc[i, 'tag'].split(',')
    for k in nums:
      if int(float(k)) in top_k_idx:
        right+=1
      else:
        pass
    correct = right/len(nums)
    print(correct)
    acc+=correct
    # print('####################################################################')
print("ACC:",acc/len(test))

(1608, 5) (1608, 13)
[ 0.  5. 42.  0.  1.] [0 0 0 0 0 0 0 0 1 0 0 0 0]
[ 0.  2. 36.  3.  0.] [0 0 0 0 0 1 0 0 0 0 0 0 1]
[ 0.  7. 58.  0.  0.] [0 0 0 0 0 0 0 0 1 0 0 1 0]
[ 0.  1. 59.  2.  2.] [0 0 0 0 1 0 0 0 0 0 0 0 0]
[ 0.  8. 42.  2.  0.] [0 0 0 0 0 1 0 0 0 0 0 0 0]
[ 0.  1. 40.  0.  4.] [0 0 0 0 0 1 0 0 0 0 0 0 0]
[ 0.  9. 38.  7.  1.] [0 0 0 1 0 0 0 0 0 0 0 0 0]
[ 0.  0. 36.  1.  2.] [0 0 0 0 0 0 0 0 1 0 0 0 0]
[ 0.  0. 41.  0.  4.] [0 0 0 0 0 0 1 0 0 0 0 0 0]
[ 0.  1. 20.  0.  1.] [0 0 0 0 0 0 0 0 0 0 0 1 0]
0.0
0.5
1.0
0.5
1.0
1.0
0.0
1.0
0.0
0.6666666666666666
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.5
0.5
0.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5
0.3333333333333333
1.0
1.0
0.0
0.3333333333333333
0.0
0.6666666666666666
0.6666666666666666
0.5
0.0
0.6666666666666666
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.5
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.6666666666666666
0.6666666666666666
1.0
1.0
1.0
0.0
1.0
0.0
1.0
0.5
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.5
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0